In [4]:
import pandas as pd

# Load the dataset from CSV
data = pd.read_csv('/content/llm data 2.0.csv')

# Check the first few rows of the dataset
print(data.head())


                                            question  \
0  How can I create a new Opportunity in Sales Cl...   
1  How do I track the sales stage of an Opportunity?   
2   How do I link Contacts to Opportunities in Sa...   
3        How can I forecast my sales in Sales Cloud?   
4       How do I create a quote from an Opportunity?   

                                              answer user_type  
0  1. Navigate to the "Opportunities" tab.2. Clic...  End User  
1  1. Open the Opportunity record.2. Find the "St...  End User  
2  1. Open the Opportunity record.2. Scroll to th...  End User  
3  1. Go to the "Forecast" tab.2. Select "Forecas...  End User  
4  1. Open the Opportunity record.2. In the "Quot...  End User  


In [5]:
print(data.columns)


Index(['question', 'answer', 'user_type'], dtype='object')


In [6]:
# Transform the data
train_data = []
for _, row in data.iterrows():
    input_text = f"question: {row['question']}"
    output_text = f"user type: {row['user_type']}, answer: {row['answer']}"
    train_data.append({'input_text': input_text, 'output_text': output_text})


data = pd.DataFrame(train_data)

# Save the transformed dataset
data.to_csv('transformed_dataset.csv', index=False)

print(data)


                                            input_text  \
0    question: How can I create a new Opportunity i...   
1    question: How do I track the sales stage of an...   
2    question:  How do I link Contacts to Opportuni...   
3    question: How can I forecast my sales in Sales...   
4    question: How do I create a quote from an Oppo...   
..                                                 ...   
160                                      question: nan   
161  question: "How do I configure and use the Even...   
162                                      question: nan   
163                                      question: nan   
164                                      question: nan   

                                           output_text  
0    user type: End User, answer: 1. Navigate to th...  
1    user type: End User, answer: 1. Open the Oppor...  
2    user type: End User, answer: 1. Open the Oppor...  
3    user type: End User, answer: 1. Go to the "For...  
4    user type: En

In [7]:

train_data = []

for _, row in data.iterrows():
    # Use the transformed column names
    input_text = row['input_text']
    output_text = row['output_text']

    # Append the preprocessed input-output pair to train_data
    train_data.append({'input': input_text, 'output': output_text})

# Create a DataFrame to inspect
train_df = pd.DataFrame(train_data)
print(train_df.head())



                                               input  \
0  question: How can I create a new Opportunity i...   
1  question: How do I track the sales stage of an...   
2  question:  How do I link Contacts to Opportuni...   
3  question: How can I forecast my sales in Sales...   
4  question: How do I create a quote from an Oppo...   

                                              output  
0  user type: End User, answer: 1. Navigate to th...  
1  user type: End User, answer: 1. Open the Oppor...  
2  user type: End User, answer: 1. Open the Oppor...  
3  user type: End User, answer: 1. Go to the "For...  
4  user type: End User, answer: 1. Open the Oppor...  


In [10]:

!huggingface-cli login
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B")
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `llm_model` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_mode

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [11]:
!pip install datasets
from datasets import Dataset

# Convert the DataFrame to a Hugging Face dataset
train_dataset = Dataset.from_pandas(train_df)

# Add padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['input'], padding="max_length", truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True)

# Prepare input/output pairs for training
train_dataset = tokenized_train.remove_columns(["input", "output"])
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Map:   0%|          | 0/165 [00:00<?, ? examples/s]

In [19]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Split the dataset into train and validation sets
train_df, val_df = train_test_split(data, test_size=0.1)

# Convert them into Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenize the datasets
# Updated tokenize_function to access 'question' instead of 'input' and handle NaN values
def tokenize_function(examples):
    # Convert 'question' to string and replace NaN with empty string
    questions = [str(q) if not pd.isnull(q) else "" for q in examples['input_text']]
    return tokenizer(questions, padding="max_length", truncation=True, max_length=512)

# **Apply tokenization and update the datasets:**
train_dataset = train_dataset.map(tokenize_function, batched=True)  # Update train_dataset
val_dataset = val_dataset.map(tokenize_function, batched=True)  # Update val_dataset


# Set format for PyTorch
# Include 'labels' for your target variable (likely the tokenized 'output')
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/148 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [20]:
# prompt: i want to finetune the model with the dataset

from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Adjust based on your resources
    per_device_eval_batch_size=1,   # Adjust based on your resources
    num_train_epochs=3,             # Adjust as needed
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch"
)

# Update tokenize_function to use input_ids instead of question and answer
def tokenize_function(examples):
    examples['labels'] = examples['input_ids']

    return examples

# Re-tokenize the datasets using the updated tokenize_function
# Assign the results back to train_dataset and val_dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)



# Set format for PyTorch
# Include 'labels' for your target variable (likely the tokenized 'output')
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/148 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.051600,0.037587


Epoch,Training Loss,Validation Loss
1,0.051600,0.037587
2,0.013400,0.041174
3,0.011300,0.037435


TrainOutput(global_step=444, training_loss=0.041179545832009196, metrics={'train_runtime': 10109.3898, 'train_samples_per_second': 0.044, 'train_steps_per_second': 0.044, 'total_flos': 1327340156092416.0, 'train_loss': 0.041179545832009196, 'epoch': 3.0})

In [21]:
# Save the model and tokenizer
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/tokenizer.json')

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the fine-tuned model and tokenizer
model_path = "./fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)



In [24]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")

def ask_model(question):
    # Format the input question
    input_text = f"question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    # Generate the model's response
    outputs = model.generate(
        **inputs,
        max_length=200,
        num_beams=5,            # Use beam search for more coherent output
        early_stopping=True,    # Stop generating once it's likely done
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract user type and answer from the response
    try:
        if "user type:" in response and "answer:" in response:
            user_type_start = response.index("user type:") + len("user type:")
            answer_start = response.index("answer:") + len("answer:")

            user_type = response[user_type_start:response.index("answer:")].strip()
            answer = response[answer_start:].strip()
        else:
            user_type = "Unknown"
            answer = response.strip()
    except ValueError:
        user_type = "Unknown"
        answer = response.strip()

    return user_type, answer

# Example usage
question = "how do i login on salesforce?"
user_type, answer = ask_model(question)
print(f"Predicted User Type: {user_type}")
print(f"Answer: {answer}")




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Predicted User Type: Unknown
Answer: question: how do i login on salesforce?
